# Hands-on session 2 - Flow-based models
## Generative Modeling Summer School 2023

#### Instructions on how to use this notebook:

This notebook is hosted on ``Google Colab``. To be able to work on it, you have to create your own copy. Go to *File* and select *Save a copy in Drive*.

You can also avoid using ``Colab`` entirely, and download the notebook to run it on your own machine. If you choose this, go to *File* and select *Download .ipynb*.

The advantage of using **Colab** is that you can use a GPU. You can complete this assignment with a CPU, but it will take a bit longer. Furthermore, we encourage you to train using the GPU not only for faster training but also to get experience with this setting. This includes moving models and tensors to the GPU and back. This experience is very valuable because, for various models and large datasets (like large CNNs for ImageNet, or Transformer models trained on Wikipedia), training on GPU is the only feasible way.

The default ``Colab`` runtime does not have a GPU. To change this, go to *Runtime - Change runtime type*, and select *GPU* as the hardware accelerator. The GPU that you get changes according to what resources are available at the time, and its memory can go from 5GB to around 18GB if you are lucky. If you are curious, you can run the following in a code cell to check:

```sh
!nvidia-smi
```

Note that despite the name, ``Google Colab`` does not support collaborative work without issues. When two or more people edit the notebook concurrently, only one version will be saved. You can choose to do group programming with one person sharing the screen with the others or make multiple copies of the notebook to work concurrently.

**Submission:** Please bring your (partial) solution to the hands-on session. Then you can discuss it with the intructors and your colleagues.

In [ ]:
!nvidia-smi

## Introduction

In this assignment, we are going to implement a Flow-based Model (flows for short). A flow-based model is a likelihood-based deep generative model that is parameterized by invertible neural networks and utilizes the change-of-variable formula. The idea of using flows was presented in multiple papers, e.g.:
- [Dinh, Laurent, Jascha Sohl-Dickstein, and Samy Bengio. "Density estimation using real nvp." arXiv preprint arXiv:1605.08803 (2016).](https://arxiv.org/abs/1605.08803)
- [Kingma, Durk P., and Prafulla Dhariwal. "Glow: Generative flow with invertible 1x1 convolutions." Advances in neural information processing systems 31 (2018).](https://proceedings.neurips.cc/paper/2018/hash/d139db6a236200b21cc7f752979132d0-Abstract.html)

You can read more about ARMs in Chapter 3 of the following book:
- [Tomczak, J.M., "Deep Generative Modeling", Springer, 2022](https://link.springer.com/book/10.1007/978-3-030-93158-2)

In particular, the goals of this assignment are the following:

- Understand how flows are formulated
- Implement flows using PyTorch
- Train and evaluate a model for image data

### Theory behind flows

Flows are probabilistic models that utilize the change-of-variable formula:
$$
p(\mathbf{x}) = \pi(\mathbf{z} = f^{-1}(\mathbf{x}))\ |\mathbf{J}_{f}(\mathbf{x})|^{-1} ,
$$
where $\pi(\cdot)$ is the *base* distribution (e.g., standard Gaussian), and $\mathbf{J}_{f}(\mathbf{x})$ is the Jacobian matrix

As a result, we can calculate the likelihood function explicitly. However, to do so, we need to pick such transformations $f$ that fulfill two requirements:

- they are **invertible**;
- the Jacobian matrix for them is **easily computable**.

The first successful implementation of $f$ that fulfills both requirements was RealNVP parameterized by coupling layers (Dinh et al., 2016).

## IMPORTS

In [ ]:
# DO NOT REMOVE!
import os

import numpy as np
import matplotlib.pyplot as plt

import torch

from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

import torchvision
from torchvision.datasets import MNIST

In [ ]:
# Check if GPU is available and determine the device
if torch.cuda.is_available():
  device = 'cuda'
else:
  device = 'cpu'

print(f'The available device is {device}')

In [ ]:
# mount drive: WE NEED IT FOR SAVING IMAGES!
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
# PLEASE CHANGE IT TO YOUR OWN GOOGLE DRIVE!
images_dir = '/content/gdrive/My Drive/Colab Notebooks/Results/'

## Auxiliary functions

Let us define some useful log-distributions:

In [ ]:
# DO NOT REMOVE
PI = torch.from_numpy(np.asarray(np.pi))
EPS = 1.e-5


def log_categorical(x, p, num_classes=256, reduction=None, dim=None):
    x_one_hot = F.one_hot(x.long(), num_classes=num_classes)
    log_p = x_one_hot * torch.log(torch.clamp(p, EPS, 1. - EPS))
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_bernoulli(x, p, reduction=None, dim=None):
    pp = torch.clamp(p, EPS, 1. - EPS)
    log_p = x * torch.log(pp) + (1. - x) * torch.log(1. - pp)
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_normal_diag(x, mu, log_var, reduction=None, dim=None):
    D = x.shape[1]
    log_p = -0.5 * D * torch.log(2. * PI) - 0.5 * log_var - 0.5 * torch.exp(-log_var) * (x - mu)**2.
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p


def log_standard_normal(x, reduction=None, dim=None):
    D = x.shape[1]
    log_p = -0.5 * D * torch.log(2. * PI) - 0.5 * x**2.
    if reduction == 'mean':
        return torch.mean(log_p, dim)
    elif reduction == 'sum':
        return torch.sum(log_p, dim)
    else:
        return log_p

## Implementing flows

The goal of this assignment is to implement one class:
- `RealNVP`: this class implements flows using coupling layers.

### RealNVP

The RealNVP is parameterized by coupling layer and allows efficient calculating of the Jacobian-determinant. Please remember that we must operate on continuous variables, thus, we must pick a proper base distribution (e.g., Gaussian).

In [ ]:
# YOUR CODE GOES HERE
# NOTE: The class must containt the following functions: 
# (i) sample
# (ii) log_prob
# (iii) log_base (i.e., log \pi(z))
# (iv) the coupling layer 
# (v) the invertible transformation f (i.e., a composition of coupling layers and permutation layers)

class RealNVP(nn.Module):
    def __init__(self, ): # ADD APPROPRIATE ATTRIBUTES
        super(RealNVP, self).__init__()
        
        # FILL IN
        pass

    def log_base(self, x):
        # FILL IN
        # output: the probability \pi(f^-1(x))
        pass
    
    def sample_base(self, batch_size=32):
        # FILL IN
        # output: a sample from \pi(z)
        pass

    def coupling(self, x, index, forward=True):
        # FILL IN
        # this is the coupling layer
        pass

    def permute(self, x):
        return x.flip(1)

    def f(self, x, forward=True):
        # FILL IN
        # HINT: the `forward` flag determines the direction (True: x->z, False: z->x)
        pass

    def sample(self, batchSize):
        # FILL IN
        # output: a sample from p(x)
        # HINT: the sampling process is two-step: (i) z ~ \pi(z), (ii) x = f(z, forward=False)
        pass

    def forward(self, x, reduction='avg'):
        # FILL IN
        # output: the log-probability that is either averaged or summed (VERY IMPORTANT!)
        pass

Please answer the following questions:

#### Question 1

Please explain how coupling layers work and why we need permutation layers.

ANSWER: [Please fill in]

#### Question 2

Please explain how one can sample from the distribution chosen by you. Please be specific and formal (i.e., provide mathematical formulae).

ANSWER: [Please fill in]

#### Question 3

Please explain why flows calculate the exact likelihood function.

ANSWER: [Please fill in]

### Evaluation and training functions

**Please DO NOT remove or modify them.**

In [ ]:
# ==========DO NOT REMOVE==========

def evaluation(test_loader, name=None, model_best=None, epoch=None):
    # EVALUATION
    if model_best is None:
        # load best performing model
        model_best = torch.load(name + '.model')

    model_best.eval()
    loss = 0.
    N = 0.
    for indx_batch, (test_batch, _) in enumerate(test_loader):
        test_batch = test_batch.to(device)
        loss_t = model_best.forward(test_batch, reduction='sum')
        loss = loss + loss_t.item()
        N = N + test_batch.shape[0]
    loss = loss / N

    if epoch is None:
        print(f'FINAL LOSS: nll={loss}')
    else:
        print(f'Epoch: {epoch}, val nll={loss}')

    return loss


def samples_real(name, test_loader, shape=(28,28)):
    # real images-------
    num_x = 4
    num_y = 4
    x, _ = next(iter(test_loader))
    x = x.to('cpu').detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], shape)
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name+'_real_images.pdf', bbox_inches='tight')
    plt.close()


def samples_generated(name, data_loader, shape=(28,28), extra_name=''):
    x, _ = next(iter(data_loader))
    x = x.to('cpu').detach().numpy()

    # generations-------
    model_best = torch.load(name + '.model')
    model_best.eval()

    num_x = 4
    num_y = 4
    x = model_best.sample(num_x * num_y)
    x = x.to('cpu').detach().numpy()

    fig, ax = plt.subplots(num_x, num_y)
    for i, ax in enumerate(ax.flatten()):
        plottable_image = np.reshape(x[i], shape)
        ax.imshow(plottable_image, cmap='gray')
        ax.axis('off')

    plt.savefig(name + '_generated_images' + extra_name + '.pdf', bbox_inches='tight')
    plt.close()


def plot_curve(name, nll_val):
    plt.plot(np.arange(len(nll_val)), nll_val, linewidth='3')
    plt.xlabel('epochs')
    plt.ylabel('nll')
    plt.savefig(name + '_nll_val_curve.pdf', bbox_inches='tight')
    plt.close()

In [ ]:
# ==========DO NOT REMOVE==========

def training(name, max_patience, num_epochs, model, optimizer, training_loader, val_loader, shape=(28,28)):
    nll_val = []
    best_nll = 1000.
    patience = 0

    # Main loop
    for e in range(num_epochs):
        # TRAINING
        model.train()
        for indx_batch, (batch, _) in enumerate(training_loader):
            batch = batch.to(device)
            loss = model.forward(batch, reduction='mean')

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validation
        loss_val = evaluation(val_loader, model_best=model, epoch=e)
        nll_val.append(loss_val)  # save for plotting

        if e == 0:
            print('saved!')
            torch.save(model, name + '.model')
            best_nll = loss_val
        else:
            if loss_val < best_nll:
                print('saved!')
                torch.save(model, name + '.model')
                best_nll = loss_val
                patience = 0

                samples_generated(name, val_loader, shape=shape, extra_name="_epoch_" + str(e))
            else:
                patience = patience + 1

        if patience > max_patience:
            break

    nll_val = np.asarray(nll_val)

    return nll_val

### Setup

**NOTE: *Please comment your code! Especially if you introduce any new variables (e.g., hyperparameters).***

In the following cells, we define `transforms` for the dataset. Next, we initialize the data, a directory for results and some fixed hyperparameters.

In [ ]:
# PLEASE DEFINE APPROPRIATE TRANFORMS FOR THE DATASET
# (If you don't see any need to do that, then you can skip this cell)
# HINT: Please prepare your data accordingly to your chosen distribution in the decoder 
transforms_train = torchvision.transforms.Compose( #FILL IN
                                                  )

transforms_test = torchvision.transforms.Compose( #FILL IN
                                                  )

Please do not modify the code in the next cell.

In [ ]:
# ==========DO NOT REMOVE==========
#-dataset
dataset = MNIST('/files/', train=True, download=True,
                      transform=transforms_train
                )

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [50000, 10000], generator=torch.Generator().manual_seed(14))

test_dataset = MNIST('/files/', train=False, download=True,
                      transform=transforms_test
                     )
#-dataloaders
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

#-creating a dir for saving results
name = 'realnvp'
result_dir = images_dir + 'results/' + name + '/'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)

#-hyperparams (please do not modify them!)
num_epochs = 1000 # max. number of epochs
max_patience = 20 # an early stopping is used, if training doesn't improve for longer than 20 epochs, it is stopped

In the next cell, please initialize the model. Please remember about commenting your code!

In [ ]:
# BASIC HYPERPARAMETERS
D = 784   # input dimension

# model definition
# YOUR CODE COMES HERE:
# FILL IN ANY OTHER HYPERPARAMS YOU WANT TO USE

In [ ]:
# INIT YOUR VAE (PLEASE CALL IT model)
# AN EXAMPLE: model = RealNVP(...)

# FILL IN

model.to(device)

Please initialize the optimizer

In [ ]:
# PLEASE DEFINE YOUR OPTIMIZER
lr = 1e-3 # learning rate (PLEASE CHANGE IT AS YOU WISH!)
optimizer = torch.optim.Adamax([p for p in model.parameters() if p.requires_grad == True], lr=lr)

#### Question 4

Please explain the choice of the optimizer, and comment on the choice of the hyperparameters (e.g., the learing reate value).

ANSWER: [Please fill in]

### Training and final evaluation

In the following two cells, we run the training and the final evaluation.

In [ ]:
# ==========DO NOT REMOVE OR MODIFY==========
# Training procedure
nll_val = training(name=result_dir + name, max_patience=max_patience, 
                   num_epochs=num_epochs, model=model, optimizer=optimizer,
                   training_loader=train_loader, val_loader=val_loader,
                   shape=(28,28))

In [ ]:
# ==========DO NOT REMOVE OR MODIFY==========
# Final evaluation
test_loss = evaluation(name=result_dir + name, test_loader=test_loader)
f = open(result_dir + name + '_test_loss.txt', "w")
f.write(str(test_loss))
f.close()

samples_real(result_dir + name, test_loader)
samples_generated(result_dir + name, test_loader, extra_name='_FINAL')

plot_curve(result_dir + name, nll_val)

### Results and discussion

After a successful training of your model, we would like to ask you to present your data and analyze it. Please answer the following questions.

#### Question 5

Please select the real data, and the final generated data and include them in this report. Please comment on the following:
- Do you think the model was trained properly by looking at the generations? Please motivate your answer well.
- Compared to the previous assignment, could you say whether this model works better (or worse)? Why?

ANSWER: [Please fill in]

#### Question 6

Please include the plot of the likelihood function. Please comment on the following:
- Is the training of your RealNVP stable or unstable? Why?
- What is the influence of the optimizer on your model? Do the hyperparameter values of the optimizer important and how do they influence the training? Motivate well your answer (e.g., run the script with more than one learning rate and present two plots here).

ANSWER: [Please fill in]